In [2]:
import pandas as pd
import scipy.constants
from scipy.optimize import curve_fit
from scipy.integrate import quad
from sklearn.metrics import r2_score
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib.transforms as transforms
from matplotlib.ticker import LogFormatter
import os
import fnmatch
%matplotlib qt
# %matplotlib inline
import datetime as dt
pd.set_option('display.max_rows', 100) 

## Reading in the data and preparing DataFrames

#### MID

In [3]:
#read in the data
mid = pd.read_csv("MID, EGA in.tsv", sep="\t", skiprows=lambda x: x <=11,  on_bad_lines="skip")
mid.drop([0,1], inplace=True)
print(mid.describe())
#bring the time data into a new df
time = pd.DataFrame(mid.iloc[:,0::2]) #- get every second column
print(time.describe())
#add elapsed time columns
def elapsed_time(df):
    itr = 0
    counter = len(df.columns)
    masses = [2,4,5.5,12,15,16,18,28,32,40,44]
    for i in df:
        timestamp = pd.to_datetime(df[i], format="%Y/%m/%d %H:%M:%S.%f")
        runtime = (timestamp-timestamp[2]).dt.total_seconds()
        colname = "Elapsed time, mass {}".format(masses[itr]) 
        df.insert(counter, colname, runtime)
        itr += 1
        counter += 1
    return(df)
elapsed_time(time)

time_sliced = time.iloc[:,11:].fillna(0)

#convert elapsed times in time-dataframe back to a timestamp
def h2_time(df1,df2):
    startdate = pd.to_datetime(df1.iloc[2,0],format="%Y/%m/%d %H:%M:%S.%f")
    dic= {}
    itr = 0
    masses = [2,4,12,16,18,28,32,40,44]
    key = "Timestamp, mass {}".format(masses[itr])
    for label,content in df2.items():
        key = "Timestamp, mass {}".format(masses[itr])
        #.strftime('%d-%m-%Y %H:%M:%S') not used
        for row in content:
            dic.setdefault(key,[]).append((startdate+dt.timedelta(milliseconds=row*1000)))
        itr += 1
    new_df = pd.DataFrame.from_dict(dic)
    return(new_df)

new_df = h2_time(time,time_sliced)
new_df


       mass 2.0_value  mass 4.0_value  mass 12.0_value  mass 16.0_value  \
count    7.920000e+02    7.920000e+02     7.920000e+02     7.920000e+02   
mean     7.038372e-12    3.042490e-13     2.412759e-13     5.531328e-13   
std      1.770834e-12    2.152121e-14     5.999172e-14     3.817115e-13   
min      6.594000e-12    2.645000e-13     0.000000e+00     2.138000e-13   
25%      6.875075e-12    2.951750e-13     2.405500e-13     5.336750e-13   
50%      6.944450e-12    3.030000e-13     2.523000e-13     5.485500e-13   
75%      7.054375e-12    3.107000e-13     2.638250e-13     5.631000e-13   
max      5.637730e-11    6.956000e-13     7.691000e-13     1.067000e-11   

       mass 18.0_value  mass 28.0_value  mass 32.0_value  mass 40.0_value  \
count     7.920000e+02     7.920000e+02     4.800000e+01     6.000000e+00   
mean      4.692543e-13     2.339114e-13     2.059167e-13     4.268833e-13   
std       3.383357e-14     9.213018e-14     1.040909e-13     4.120944e-13   
min       4.0100

,"Timestamp, mass 2","Timestamp, mass 4","Timestamp, mass 12","Timestamp, mass 16","Timestamp, mass 18","Timestamp, mass 28","Timestamp, mass 32"
0,2022-11-15 14:16:16.895,2022-11-15 14:16:16.895,2022-11-15 14:16:16.895,2022-11-15 14:16:16.895,2022-11-15 14:16:16.895,2022-11-15 14:16:16.895,2022-11-15 14:16:16.895
1,2022-11-15 14:16:19.218,2022-11-15 14:16:19.218,2022-11-15 14:16:19.218,2022-11-15 14:16:19.219,2022-11-15 14:16:19.218,2022-11-15 14:16:19.469,2022-11-15 14:16:16.895
2,2022-11-15 14:16:21.541,2022-11-15 14:16:21.542,2022-11-15 14:16:21.542,2022-11-15 14:16:21.542,2022-11-15 14:16:21.541,2022-11-15 14:16:21.791,2022-11-15 14:16:16.895
3,2022-11-15 14:16:24.115,2022-11-15 14:16:24.115,2022-11-15 14:16:24.115,2022-11-15 14:16:24.616,2022-11-15 14:16:16.895,2022-11-15 14:16:16.895,2022-11-15 14:16:16.895
4,2022-11-15 14:16:26.437,2022-11-15 14:16:26.437,2022-11-15 14:16:26.438,2022-11-15 14:16:26.938,2022-11-15 14:16:16.895,2022-11-15 14:16:16.895,2022-11-15 14:16:16.895
...,...,...,...,...,...,...,...
787,2022-11-15 14:46:44.955,2022-11-15 14:46:44.955,2022-11-15 14:46:44.955,2022-11-15 14:46:45.456,2022-11-15 14:16:16.895,2022-11-15 14:16:16.895,2022-11-15 14:16:16.895
788,2022-11-15 14:46:47.277,2022-11-15 14:46:47.277,2022-11-15 14:46:47.278,2022-11-15 14:46:47.778,2022-11-15 14:16:16.895,2022-11-15 14:16:16.895,2022-11-15 14:16:16.895
789,2022-11-15 14:46:49.600,2022-11-15 14:46:49.600,2022-11-15 14:46:49.600,2022-11-15 14:46:50.101,2022-11-15 14:16:16.895,2022-11-15 14:16:16.895,2022-11-15 14:16:16.895
790,2022-11-15 14:46:51.922,2022-11-15 14:46:51.922,2022-11-15 14:46:51.923,2022-11-15 14:46:52.423,2022-11-15 14:16:16.895,2022-11-15 14:16:16.895,2022-11-15 14:16:16.895


#### Vaclogger

In [4]:
#reading the Emission current, pressures
vaclog=pd.read_csv("vaclog", sep="\t")
vaclog.head()
#adding an elapsed time column
#making the column into a datetime object
vac_timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(vac_timestamps-vac_timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
#drop index column (not necessary)
# vaclog.drop(index=vaclog.index[0], 
#         axis=0, 
#         inplace=True)
vaclog["Time"] = vac_timestamps.dt.strftime('%d-%m-%Y %H:%M:%S')
vaclog["Time"] = pd.to_datetime(vaclog["Time"],format='%d-%m-%Y %H:%M:%S')

vaclog.head()

,Live comments,Time,Elapsed time,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,I_emission,I_grid
0,NaN,2022-11-15 14:15:05,0.0,-0.067249,5.610000e-10,2.960000e-10,4.996000e-09,4.549000e-07,0.000485,260.0,-5.163,4293.361,NaN,NaN
1,NaN,2022-11-15 14:15:12,7.0,-0.067276,5.610000e-10,2.960000e-10,4.996000e-09,4.549000e-07,0.000488,260.0,-5.050,4293.361,NaN,NaN
2,NaN,2022-11-15 14:15:19,14.0,-0.066733,5.610000e-10,2.960000e-10,4.996000e-09,4.549000e-07,0.000481,260.0,-5.163,4293.473,NaN,NaN
3,NaN,2022-11-15 14:15:26,21.0,-0.066832,5.620000e-10,2.960000e-10,4.996000e-09,4.549000e-07,0.000497,259.0,-5.163,4293.585,NaN,NaN
4,NaN,2022-11-15 14:15:33,28.0,-0.066978,5.620000e-10,2.970000e-10,4.996000e-09,4.549000e-07,0.000487,260.0,-5.163,4293.585,NaN,NaN


#### HiVolta log

In [5]:
#Reading in the HiVolta measurement file 
hv = pd.read_csv("hv log", sep=",")
#hv.drop(columns=["Vmon1","Vmon2","Vmon3","Vmon4","Vmon5","Vmon6","Vmon7","Vmon8","Imon3","Imon4","Imon5","Imon6","Imon7","Imon8","Comments1"], inplace=True)
hv["hv_grid"]=[element * 1e-6 for element in hv["IMon1"]]
hv["em"]=[element * 1e-6 for element in hv["IMon2"]]
hv.drop(hv.columns[hv.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
hv["em"]=abs(hv["em"])
hv["hv_grid"]=abs(hv["hv_grid"])

#adding a datetime column
datetime = pd.to_datetime(hv['Date'] + ' ' + hv['Time'],format="%d/%m/%Y %H:%M:%S.%f")
datetime = datetime.dt.strftime('%d-%m-%Y %H:%M:%S')
datetime = pd.to_datetime(datetime, format='%d-%m-%Y %H:%M:%S')
hv.insert(0,"datetime",datetime)
    
#making the column into a datetime object
hv_timestamps=pd.to_datetime(hv["Time"],format="%H:%M:%S.%f")
runtime=(hv_timestamps-hv_timestamps[0]).dt.total_seconds()    
hv.insert(2,"Elapsed time",runtime)

hv.head()



,datetime,Date,Elapsed time,Time,VMon1,VMon2,VMon3,VMon4,VMon5,VMon6,...,IMon2,IMon3,IMon4,IMon5,IMon6,IMon7,IMon8,Comment,hv_grid,em
0,2022-11-15 14:17:27,15/11/2022,0.000,14:17:27.045,0.02,0.08,0.02,0.08,0.0,0.0,...,0.0011,0.0107,0.0074,0.0078,-0.0152,-0.019,-0.0148,no emission,9.000000e-10,1.100000e-09
1,2022-11-15 14:17:28,15/11/2022,1.187,14:17:28.232,0.02,0.08,0.02,0.08,0.0,0.0,...,0.0011,0.0107,0.0074,0.0078,-0.0152,-0.019,-0.0148,no emission,9.000000e-10,1.100000e-09
2,2022-11-15 14:17:29,15/11/2022,2.373,14:17:29.418,0.02,0.08,0.02,0.08,0.0,0.0,...,0.0011,0.0105,0.0074,0.0151,-0.0152,-0.019,-0.0148,no emission,9.000000e-10,1.100000e-09
3,2022-11-15 14:17:30,15/11/2022,3.560,14:17:30.605,0.02,0.08,0.02,0.08,0.0,0.0,...,0.0011,0.0099,0.0074,0.0149,-0.0152,-0.019,-0.0148,no emission,9.000000e-10,1.100000e-09
4,2022-11-15 14:17:31,15/11/2022,4.748,14:17:31.793,0.02,0.08,0.02,0.08,0.0,0.0,...,0.0011,0.0099,0.0074,0.0139,-0.0152,-0.019,-0.0148,no emission,9.000000e-10,1.100000e-09


## Data processing

In [6]:
#print vaclog comments
print(pd.unique(vaclog["Live comments"]))

[nan]


In [7]:
#print hv comments
print(pd.unique(hv["Comment"]))
hv_comm = hv.loc[:,["datetime","Comment"]]
hv_comm.dropna(inplace=True)
print(hv_comm)
hv_comm.index

['no emission' nan 'emission on' 'grid repelling' 'grid transparent again'
 'grid repelling again' 'grid transparent & EGA turned' 'end']
                datetime                        Comment
0    2022-11-15 14:17:27                    no emission
1    2022-11-15 14:17:28                    no emission
2    2022-11-15 14:17:29                    no emission
3    2022-11-15 14:17:30                    no emission
4    2022-11-15 14:17:31                    no emission
71   2022-11-15 14:18:53                    emission on
72   2022-11-15 14:18:54                    emission on
73   2022-11-15 14:18:55                    emission on
74   2022-11-15 14:18:56                    emission on
534  2022-11-15 14:28:07                 grid repelling
535  2022-11-15 14:28:08                 grid repelling
536  2022-11-15 14:28:09                 grid repelling
537  2022-11-15 14:28:10                 grid repelling
538  2022-11-15 14:28:11                 grid repelling
730  2022-11-15 14:32:

Int64Index([   0,    1,    2,    3,    4,   71,   72,   73,   74,  534,  535,
             536,  537,  538,  730,  731,  732,  733,  809,  811,  813,  815,
            1250, 1251, 1252, 1253, 1254, 1476, 1477, 1478, 1479],
           dtype='int64')

#### CernOx temperature R-T conversion

In [8]:
#Temperature curve for CERNOX - for temp stability
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]

#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


## Checking injection volume pressure drop linearity

In [9]:
#defining a function for the straight line
def fit(x,a,b):
    return a*x + b
#curve fit for the data
y_val = vaclog["injection 1mbar"].values
x_val = (vaclog["Elapsed time"]/3600).values
params, cov = curve_fit(fit,x_val,y_val)
#straight line parameters
a, b = params
print(f"Fitted line: y = {a:.2f}*x+{b:.2f}")

Fitted line: y = -0.00*x+0.00


#### Data preparation for calculating max desorption yield (w/o recycling)

In [10]:
#Filtering background signal

#Find the timestamp where emission was turned on and off
#print("Result 1:",hv.loc[hv["Comment"]=="grid transparent"])
#print("Result 2:", hv.loc[hv["Comment"]=="grid repelling"])
#print("Result 3:", hv.loc[hv["Comment"]=="electron activity grid repelling"])

x=pd.Timestamp(2023, 1, 12, 17 ,23, 58)
y=pd.Timestamp(2023, 1, 12, 17, 28, 43)
z=pd.Timestamp(2023, 1, 12, 17, 20, 13)

#then find the time value closest to it in the MID scan file 
#print("\n","Result 4:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - x).abs().argsort(),:])
#print("\n","Result 5:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - y).abs().argsort(),:])
#print("\n","Result 6:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - z).abs().argsort(),:])

#take a mass with a signal known to be in the electronics background
elec_bck = 0

#apply elec background correction to h2 data
mid["mass 2, corr"] = mid["mass 2.0_value"] - elec_bck

##apply base pressure correction to mass 2.0 data
#dyn_bck = np.mean(mid["mass 2, corr"][12:56])
#print("dyn_ bck",f"{dyn_bck}")

#subtracting dyn bck like this deletes useful data
#mid["mass 2, corr"] = mid["mass 2, corr"] - dyn_bck

#remove values below 0:
#mid["mass 2, corr"].clip(0, inplace=True) 

#From the signal, get the partial pressures 
h2_sens = 2.13e-2 #A/mbar
h2_equiv = 2.49 #H2 equivalent pressures from N2 reading

#account for the thermal transpiration factor
T2 = 294
T1 = 4.2
T_coef = np.sqrt(T2/T1)

#apply correction factors 
#p_h2 = (mid["mass 2, corr"][91:136]*h2_equiv)/(h2_sens*T_coef) 
#mid.insert(22,"p_h2",p_h2)
#print(p_h2)

#baseline and emission pp for partial pressure delta
#em_h2 = mid["p_h2"][91:136] #hardcoded values for "esd on"
#base_h2 = mid["p_h2"][12:56] #hardcoded values for "esd off"
#print(base_h2)

In [11]:
#NOT USED!

# #non hard coded values for "esd on/off"
# #find the local maxima
# print(mid.p_h2.values)
# mid['ESD max'] = mid.iloc[argrelextrema(mid.p_h2.values, np.greater_equal, order=5)[0]]['p_h2']
# mid['ESD min'] = mid.iloc[argrelextrema(mid.p_h2.values, np.less_equal, order=5)[0]]['p_h2']

# print(mid["ESD max"])
# print(mid["ESD min"])
# #take the 10 next values from max - define as "em on"


# #take the 10 next values from min - define as "em off"

# Apparent desorption yield calculation for H2
### Formula for the cryo setup

$$
\eta_{app}=\frac{C_{comb}\cdot (p_{n,EM} - \overline{p}_{base}) \cdot q_e}{k_B\cdot T \cdot I_{n}}
$$

T here is @RT and therefore thermal transpiration compensation should be applied afterwards to $\Delta p$.

$C_{comb}$ takes into account the conductance from the sample via the copper tube for H2 and the pumping speed for turbo pump

In [12]:
#calculation of delta_p-s
#dp1 = (np.mean(p_h2)-np.mean(base_h2))*100
#print(dp1)

#Constants
k_B = scipy.constants.value("Boltzmann constant")
T = 4.2
q_e = scipy.constants.value("elementary charge")
c_h2 = 37.91/1000
s_h2 = 42/1000
x_h2 = 1/((1/c_h2)+(1/s_h2))
print(x_h2)

#Emission current
I_em = np.nanmean(hv["em"].nlargest(30))
print(I_em)


0.01992516581153798
1.5633333333333331e-06


In [13]:
#Desorption yield
#eta_h2 = (x_h2*dp1*q_e)/(k_B*T*I_em)
#eta_co = (c_co*(dp1-dp2)*q_e)/(k_B*T*I_em)
#print("The desorption yield for H2 [mol/el]:",eta_h2,"\n",)#""The desorption yield for CO2 [mol/el]:",eta_co2)

#### Data manipulation for point-by-point ESD

In [14]:
#make a sub-df from mid that includes only mass 2 data
h2 = mid.loc[:, ["mass 2.0_time", "mass 2.0_value"]]
#rename columns
h2 = h2.rename(columns={"mass 2.0_time": "datetime","mass 2.0_value": "Ion current"}).reset_index(drop=True)

print(h2)

                    datetime   Ion current
0    2022/11/15 14:16:12.249  7.131000e-12
1    2022/11/15 14:16:14.572  7.066500e-12
2    2022/11/15 14:16:16.895  7.137600e-12
3    2022/11/15 14:16:19.218  7.055800e-12
4    2022/11/15 14:16:21.541  6.972800e-12
..                       ...           ...
787  2022/11/15 14:46:40.058  7.014300e-12
788  2022/11/15 14:46:42.381  6.998600e-12
789  2022/11/15 14:46:44.703  7.054300e-12
790  2022/11/15 14:46:47.026  7.068100e-12
791  2022/11/15 14:46:49.348  7.031000e-12

[792 rows x 2 columns]


In [15]:
# Convert the 'time' column to datetime index for both dataframes
hv.index = (pd.to_datetime(hv['datetime']).dt.round(freq="S"))
h2.index = (pd.to_datetime(h2['datetime']).dt.round(freq="S"))

# Drop the original "datetime" columns since they are no longer needed
hv.drop('datetime', axis=1, inplace=True)
h2.drop('datetime', axis=1, inplace=True)

# Ensure the index of hv DataFrame is unique and sorted
hv = hv[~hv.index.duplicated(keep='first')]
hv = hv.sort_index()
hv = hv.iloc[:-1]
# Ensure the index of h2 DataFrame is unique and sorted
h2 = h2[~h2.index.duplicated(keep='first')]
h2 = h2.sort_index()

print("hv", hv[-3:])
#print("h2",h2)

# Reindex hv DataFrame to match the datetime index of h2 DataFrame
hv = hv.reindex(h2.index, method='nearest', limit=1)


# Now hv and h2 have the same datetime index and can be joined or aligned
# by joining on the index
hv_aligned = hv.merge(h2, left_index=True, right_index=True, how="inner")

print("aligned", hv_aligned)
print(hv_aligned.describe())


hv                            Date  Elapsed time          Time  VMon1   VMon2  \
datetime                                                                     
2022-11-15 14:46:53  15/11/2022      1766.529  14:46:53.574   0.12  100.06   
2022-11-15 14:46:54  15/11/2022      1767.723  14:46:54.768   0.10  100.06   
2022-11-15 14:46:55  15/11/2022      1768.919  14:46:55.964   0.12  100.06   

                      VMon3    VMon4   VMon5  VMon6  VMon7  ...   IMon2  \
datetime                                                    ...           
2022-11-15 14:46:53  999.96  1000.08  1000.1    0.0    0.0  ...  1.0552   
2022-11-15 14:46:54  999.96  1000.08  1000.1    0.0    0.0  ...  1.0534   
2022-11-15 14:46:55  999.98  1000.08  1000.1    0.0    0.0  ...  1.0534   

                       IMon3    IMon4    IMon5   IMon6   IMon7   IMon8  \
datetime                                                                 
2022-11-15 14:46:53  13.8927  13.8419  13.8104 -0.0105 -0.0197 -0.0121   
2022-11-

In [16]:
#Filtering background signal for interpolated datasets


#From the signal, get the partial pressures 
h2_sens = 2.13e-2 #A/mbar
p_h2_ip = hv_aligned["Ion current"]/(h2_sens) 

#account for the thermal transpiration factor
T2 = 4.2
T1 = 293
T_coef = np.sqrt(T2/T1)
p_h2_ip = p_h2_ip*T_coef 

#insert partial pressures into the dataframe
hv_aligned.insert(19,"hydrogen pp",p_h2_ip)


In [17]:
hv_aligned.fillna(0,inplace=True)
hv_aligned["hydrogen pp"].mask(hv_aligned["hydrogen pp"].ge(6.05e-11), inplace=True)
print(hv_aligned["hydrogen pp"].describe())
hv_aligned["Elapsed time"].describe()


count    7.910000e+02
mean     3.921187e-11
std      1.312211e-12
min      3.706467e-11
25%      3.864388e-11
50%      3.903145e-11
75%      3.965088e-11
max      5.997574e-11
Name: hydrogen pp, dtype: float64


count     792.000000
mean      845.898615
std       528.793902
min         0.000000
25%       385.127250
50%       844.739000
75%      1303.377250
max      1762.935000
Name: Elapsed time, dtype: float64

#### Using polynomial regression, remove the pressure decrease from pumping

In [18]:
""" # Load the data
t = time["Elapsed time, mass 2"]
print(t.describe())
I_ion = mid["Ion current"]
print(I_ion.describe())
#The function for the ion current-time relationship is not known. 
#numpy method for creating a polynomial model    
model = np.poly1d(np.polyfit(t, I_ion, 8))
print(model(t))

rsquared = r2_score(I_ion,model(t))
print(rsquared)

# Print the original function
plt.figure(figsize=(12,6))
plt.plot(t, I_ion)

# Add labels and title
plt.xlabel('time (s)')
plt.ylabel('Ion current')
plt.title('Ion current vs time')

plt.yscale("log")
plt.show()
 """

' # Load the data\nt = time["Elapsed time, mass 2"]\nprint(t.describe())\nI_ion = mid["Ion current"]\nprint(I_ion.describe())\n#The function for the ion current-time relationship is not known. \n#numpy method for creating a polynomial model    \nmodel = np.poly1d(np.polyfit(t, I_ion, 8))\nprint(model(t))\n\nrsquared = r2_score(I_ion,model(t))\nprint(rsquared)\n\n# Print the original function\nplt.figure(figsize=(12,6))\nplt.plot(t, I_ion)\n\n# Add labels and title\nplt.xlabel(\'time (s)\')\nplt.ylabel(\'Ion current\')\nplt.title(\'Ion current vs time\')\n\nplt.yscale("log")\nplt.show()\n '

In [19]:
""" #normalise measured data to fit
x = np.linspace(5095.951000, 19700, 3095)
I_norm = I_ion/model(x)

# Plot the normalised ion current
plt.figure(figsize=(12,6))
plt.plot(t, I_norm)

# Add labels and title
plt.xlabel('time (s)')
plt.ylabel('normalised ion current (a.u.)')
plt.title('Ion current vs time')

plt.yscale("linear")
plt.show()

#now we have  """

' #normalise measured data to fit\nx = np.linspace(5095.951000, 19700, 3095)\nI_norm = I_ion/model(x)\n\n# Plot the normalised ion current\nplt.figure(figsize=(12,6))\nplt.plot(t, I_norm)\n\n# Add labels and title\nplt.xlabel(\'time (s)\')\nplt.ylabel(\'normalised ion current (a.u.)\')\nplt.title(\'Ion current vs time\')\n\nplt.yscale("linear")\nplt.show()\n\n#now we have  '

#### Checking injection volume pressure drop linearity

In [20]:
#defining a function for the straight line
def fit(x,a,b):
    return a*x + b
#curve fit for the data
y_val = vaclog["injection 1mbar"].values
x_val = (vaclog["Elapsed time"]/3600).values
params, cov = curve_fit(fit,x_val,y_val)
#straight line parameters
a, b = params
print(f"Fitted line: y = {a:.2f}*x+{b:.2f}")

Fitted line: y = -0.00*x+0.00


## Desorption yield point-by-point

$$
\eta_{app}=\frac{C_{comb}\cdot (p_{n,EM} - \overline{p}_{base}) \cdot q_e}{k_B\cdot T \cdot I_{n}}
$$

In [21]:
""" #partial pressure delta
#wrong methodology
p_delta = 
p_delta = p_delta.clip(lower=0)
print(p_delta)
#hv_shifted = hv_aligned.shift(periods=1)
#print(hv_shifted)
#p_delta = hv_aligned["hydrogen pp"] - hv_shifted["hydrogen pp"]
#print(hv_aligned["hydrogen pp"].describe())
#print(hv_shifted["hydrogen pp"].describe())
print(p_delta.describe()) """

' #partial pressure delta\n#wrong methodology\np_delta = \np_delta = p_delta.clip(lower=0)\nprint(p_delta)\n#hv_shifted = hv_aligned.shift(periods=1)\n#print(hv_shifted)\n#p_delta = hv_aligned["hydrogen pp"] - hv_shifted["hydrogen pp"]\n#print(hv_aligned["hydrogen pp"].describe())\n#print(hv_shifted["hydrogen pp"].describe())\nprint(p_delta.describe()) '

In [22]:
""" #constants are already defined
#emission current and H2 partial pressures are now available at the same sampling rates
print(hv_aligned.describe())
I_n = hv_aligned["em"].replace(0, np.nan).dropna()

I_n.where(hv_aligned["em"] >= 5e-7, np.nan, inplace=True)
#pd.df.where(cond= "Where cond is True, keep the original value. Otherwise, replace with NaN")
print(I_n.describe())

eta = ((x_h2*p_delta*q_e)/(k_B*T*I_n)).dropna()
eta = eta.loc[eta.index[1]:]
print("desorption yields point-by-point:","\n", eta) """

' #constants are already defined\n#emission current and H2 partial pressures are now available at the same sampling rates\nprint(hv_aligned.describe())\nI_n = hv_aligned["em"].replace(0, np.nan).dropna()\n\nI_n.where(hv_aligned["em"] >= 5e-7, np.nan, inplace=True)\n#pd.df.where(cond= "Where cond is True, keep the original value. Otherwise, replace with NaN")\nprint(I_n.describe())\n\neta = ((x_h2*p_delta*q_e)/(k_B*T*I_n)).dropna()\neta = eta.loc[eta.index[1]:]\nprint("desorption yields point-by-point:","\n", eta) '

In [23]:
""" #slicing eta to match hv_aligned["Elapsed time"]:
# Slice x using the index of y
hv_downsampled = hv_aligned.loc[eta.index, :] """

' #slicing eta to match hv_aligned["Elapsed time"]:\n# Slice x using the index of y\nhv_downsampled = hv_aligned.loc[eta.index, :] '

## Electron dose


Find:

$$
Q = \int_{t_1}^{t_2} I \,dt
$$

Where $t_2$ and $t_1$ are the timestamps at which an emission current reading was taken


In [24]:
""" #Integrating the emission current
def integrand(t, i):
    return i*t

i = I_n.dropna().reset_index() 

#for testing:
#t2_2 = (hv_aligned["Elapsed time"].diff(periods=1).fillna(0))

#adding an elapsed time column
timestamp3=[]
for j in hv["Time"]:
    timestamp3.append(j)
    
#making the column into a datetime object
timestamps3=pd.to_datetime(i["datetime"],format="%H:%M:%S")
runtime3=(timestamps3-timestamps3[0]).dt.total_seconds()    
i.insert(2,"Elapsed time",runtime3)

#define the n, n-1 difference in a variable "t2"
t2 = (i["Elapsed time"].diff(periods=1).fillna(0))

#replace all values from t2 that are further away than 50% of the median of the sampling rate (.. > 9) with the median value
t2.where(t2 <= 12,6,inplace=True)
print(t2)

#integration
ar = []
for x in t2:
    itr = 0
    I = quad(integrand, 0, x, args=(i["em"][itr]))
    ar.append(I[0])
    itr += 1
    #if previous value was not NaN, replace the value with the median of elapsed time
    #if 
     """

' #Integrating the emission current\ndef integrand(t, i):\n    return i*t\n\ni = I_n.dropna().reset_index() \n\n#for testing:\n#t2_2 = (hv_aligned["Elapsed time"].diff(periods=1).fillna(0))\n\n#adding an elapsed time column\ntimestamp3=[]\nfor j in hv["Time"]:\n    timestamp3.append(j)\n    \n#making the column into a datetime object\ntimestamps3=pd.to_datetime(i["datetime"],format="%H:%M:%S")\nruntime3=(timestamps3-timestamps3[0]).dt.total_seconds()    \ni.insert(2,"Elapsed time",runtime3)\n\n#define the n, n-1 difference in a variable "t2"\nt2 = (i["Elapsed time"].diff(periods=1).fillna(0))\n\n#replace all values from t2 that are further away than 50% of the median of the sampling rate (.. > 9) with the median value\nt2.where(t2 <= 12,6,inplace=True)\nprint(t2)\n\n#integration\nar = []\nfor x in t2:\n    itr = 0\n    I = quad(integrand, 0, x, args=(i["em"][itr]))\n    ar.append(I[0])\n    itr += 1\n    #if previous value was not NaN, replace the value with the median of elapsed time\

In [25]:
""" #calculate the cumulative charge
Q_cumul = np.cumsum(ar)
Q_sum = sum(ar)
print("Total charge emitted from EGA during ~5000 seconds:", f"{Q_sum:.4E}", "C")

#cumulative nr of electrons emitted
N_e = Q_sum/q_e
Ne_cumul = Q_cumul/q_e
print("Total number of electron emitted during ~5000 seconds:", f"{N_e:.4E}", "e")
      
#To get the electron dose, divide n of electrons by sample surface area of 276 cm2, multiply by grid transmittance of 0.9
tot_dose = (N_e/276)*0.9
cumul_dose = (Ne_cumul/276)*0.9
print("Total electron dose on the sample ~5000 seconds:", f"{tot_dose:.4E}", "e-/cm^2")
      
#add these to the "i" dataframe:
i.insert(3,"Cumulative charge", Q_cumul)
i.insert(4,"Cumulative n. of electrons emitted from EGA", Ne_cumul)
i.insert(5,"Cumulative electron dose on the HL sample", cumul_dose)
i.set_index("datetime", drop=True, inplace=True)

#Ignore the first value, also, match the shape of eta
i = i.loc[i.index[1]:]
print(i)
 """

' #calculate the cumulative charge\nQ_cumul = np.cumsum(ar)\nQ_sum = sum(ar)\nprint("Total charge emitted from EGA during ~5000 seconds:", f"{Q_sum:.4E}", "C")\n\n#cumulative nr of electrons emitted\nN_e = Q_sum/q_e\nNe_cumul = Q_cumul/q_e\nprint("Total number of electron emitted during ~5000 seconds:", f"{N_e:.4E}", "e")\n      \n#To get the electron dose, divide n of electrons by sample surface area of 276 cm2, multiply by grid transmittance of 0.9\ntot_dose = (N_e/276)*0.9\ncumul_dose = (Ne_cumul/276)*0.9\nprint("Total electron dose on the sample ~5000 seconds:", f"{tot_dose:.4E}", "e-/cm^2")\n      \n#add these to the "i" dataframe:\ni.insert(3,"Cumulative charge", Q_cumul)\ni.insert(4,"Cumulative n. of electrons emitted from EGA", Ne_cumul)\ni.insert(5,"Cumulative electron dose on the HL sample", cumul_dose)\ni.set_index("datetime", drop=True, inplace=True)\n\n#Ignore the first value, also, match the shape of eta\ni = i.loc[i.index[1]:]\nprint(i)\n '

# Plotting

In [26]:
#define parameters for plotting
#annotation params
font = dict(size = "x-small", color ="green", style ="italic",rotation="vertical")

plt.rcParams["figure.figsize"] = [8,5]
plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams["axes.grid"] = True
plt.rcParams["grid.color"] = "black"
plt.rcParams["grid.linewidth"] = 0.35

formatter = md.DateFormatter('%H:%M')

# Create the "graphs" folder path
graphs_folder = os.path.join(os.getcwd(), 'graphs')

# Ensure the "graphs" folder exists, if not create it
if not os.path.exists(graphs_folder):
    os.makedirs(graphs_folder)
    

In [27]:
#plot the aligned dataframe results
plt.figure(figsize=(12,6))
plt.plot(hv_aligned["Elapsed time"],hv_aligned["Ion current"],linestyle="-",marker=".", markersize=4,label='H2 ion current')
plt.plot(hv_aligned["Elapsed time"],hv_aligned["em"],linestyle="-",marker=".", markersize=4,label='Emission current')
plt.xlabel('Elapsed time (s)')
plt.ylabel('Current (A)')
plt.yscale('log')

plt.legend(loc="upper right")
plt.title("Current evolution 1 ML tech. surface pre-injection, EGA fully in, dynamic vacuum")


plt.yscale('log')


In [28]:
#plot the h2 mid 
#seems to be too little rows?
plt.figure(figsize=(12,6))
plt.plot(hv_aligned["Elapsed time"],hv_aligned["Ion current"],linestyle="-",marker=".", markersize=4,label='H2 ion current')
plt.xlabel('Elapsed time (s)')
plt.ylabel('Ion current (A)')
plt.legend(loc="upper right")
plt.title("Ion current evolution of H2, 1 ML tech. surface pre-injection, EGA fully in, dynamic vacuum")


plt.yscale('linear')


In [29]:
#plot the aligned dataframe results
fig, ax = plt.subplots()
y_fit = fit(x_val,a,b)
ax.plot(hv_aligned["Elapsed time"],hv_aligned["em"],linestyle="-",marker=".",c="r", markersize=4,label='I_EM')
ax.set_xlim(left=140)

ax.set_xlabel('Elapsed time (s)')
ax.set_ylabel('Current (A)')
ax.set_yscale('linear')
ax.legend(loc="upper right")
ax.axes.grid(visible=None,axis="y")
ax2 = ax.twinx()
ax2.plot(hv_aligned["Elapsed time"],hv_aligned["hydrogen pp"],linestyle="-",marker=".", c="b",markersize=4,label='H2 partial pressure')
ax2.set_ylabel('Pressure (mbar)')
ax2.set_yscale('linear')
ax2.legend(loc="upper left")

plt.title("HiLumi sample ESD @4K no pre-coverage. H2 Partial pressure and emission current over time")

Text(0.5, 1.0, 'HiLumi sample ESD @4K no pre-coverage. H2 Partial pressure and emission current over time')

In [30]:
#Plotting the all the relevant MID spectrums
plt.figure(figsize=(14,8))

plt.plot(new_df["Timestamp, mass 2"],mid["mass 2.0_value"]/2.13e-2,marker=".", markersize=4,label='M2')# MBAR
# plt.plot(new_df["Timestamp, mass 44"],mid["mass 44.0_value"],marker=".", markersize=4,label='M44')

plt.xlabel('Timestamp')
plt.ylabel('Pressure (mbar)')
plt.legend(title="Mol masses", loc="upper right")
plt.title("S1 pressure from MID scan")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(14))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')

#write comments
trans  = transforms.blended_transform_factory(
    ax.transData, ax.transAxes)
for i in hv_comm.index:
    plt.text(hv_comm["datetime"][i],0.95,str(hv_comm["Comment"][i]),fontdict=font, transform=trans)


plt.savefig(r'./graphs/pressure from mid.png')
plt.show()

In [31]:
#plotting HiVolta log
plt.figure(figsize=(12,6))
plt.plot(hv["Elapsed time"],hv["hv_grid"],marker=".", markersize=4,label='Grid current')
plt.plot(hv["Elapsed time"],hv["em"],marker=".", markersize=4,label='Emission current')
plt.xlabel('Elapsed time (s)')
plt.ylabel('Current (A)')
plt.legend(loc="upper right")
plt.title("Current evolution 1 ML tech. surface pre-injection, EGA fully in, dynamic vacuum")


plt.yscale('log')


In [32]:
#Plotting BA2, BA1 pressure
plt.figure(figsize=(12,6))
plt.plot(vaclog["Time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(vaclog["Time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')
plt.xlabel('Timestamp')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("S1 Total pressure evolution")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')    
       
plt.savefig(r'./graphs/Barions.png')
plt.show()

In [33]:
## Plotting the temp evolution
plt.figure(figsize=(10,6))
plt.plot((vaclog["Time"]),polyfit(A,vaclog["T-CERNOX"]),marker=".", markersize=5,label='T-CERNOX')          
plt.xlabel('Timestamp')
plt.ylabel('Temperature (K)')
plt.legend()
plt.title("S1 Temperature evolution")
plt.yscale('linear')
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')
    
plt.savefig(r'./graphs/Temp.png')
plt.show()

In [34]:
#checking inj pressure linearity, graph
fig, ax = plt.subplots()
y_fit = fit(x_val,a,b)
ax.plot((vaclog["Elapsed time"]/3600),vaclog["injection 1mbar"],marker=".", markersize=5,label='inj volume pressure')
ax.plot((vaclog["Elapsed time"]/3600),y_fit, "--",color="red", label="Y= -ax + b fit") 
ax.legend(loc="upper right")

ax.set_xlabel('Elapsed time (h)')
ax.set_ylabel('Pressure (mbar)')
ax.set_yscale('linear')
plt.title("Injection pressure evolution 1mbar conductance gauge")

Text(0.5, 1.0, 'Injection pressure evolution 1mbar conductance gauge')